In [1]:
import os
import warnings

import polars as pl
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from scipy.special import erfc
# from math import erfc
from lets_plot import (
    LetsPlot,
    element_blank,
    theme,
    geom_point,
    geom_line,
    scale_x_log10,
    scale_y_log10,
    labs,
    scale_fill_brewer,
    scale_color_brewer,
    gggrid,
    ggplot,
    ggsize,
    lims,
    theme_classic, aes,
    scale_color_viridis,
    scale_fill_viridis,
    element_text,
    element_geom,
    scale_color_manual,
    scale_fill_manual,
    scale_fill_hue,
    scale_color_hue,
    geom_histogram,
    theme_void,
    geom_density,
)
LetsPlot.setup_html()

In [2]:
def double_exp(x, a, b, c, d):
    return a * np.exp(-x * b) + (d) * np.exp(-x * c)


def erfc_exp(t, A, tau, beta):
    return A * np.exp(-(t / tau) ** beta)


def erfc_exp(t, a, koff, ks):
    return (a 
            * np.exp(( np.square(koff) * t) / ks) 
            * erfc(np.sqrt((np.square(koff) * t) / ks))
    )

def powerlaw(x, a, b):
    return a * x ** (-b)


def exp_decay(x, a, b):
    return a * np.exp(-x * b)


def tri_exp(x, a, b, c, d, e, f):
    return a * np.exp(-x * b) + c * np.exp(-x * d) + e * np.exp(-x * f)


def quad_exp(x, a, b, c, d, e, f, g, h):
    return (
        a * np.exp(-x * b)
        + c * np.exp(-x * d)
        + e * np.exp(-x * f)
        + g * np.exp(-x * h)
    )


def penta_exp(x, a, b, c, d, e, f, g, h, i, j):
    return (
        a * np.exp(-x * b)
        + c * np.exp(-x * d)
        + e * np.exp(-x * f)
        + g * np.exp(-x * h)
        + i * np.exp(-x * j)
    )

def deleteNaN(y: np.ndarray,t: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """
    delete NaN parts of the input array and time array opened for it,
    and returns time array and values array.

    """

    t = t[~np.isnan(y)]
    y = y[~np.isnan(y)]

    return t, y

def value_fit(
    val: np.ndarray, t: np.ndarray, tmax: int, 
    eq: callable,
    delete_nan: bool = True
) -> tuple[np.ndarray, np.ndarray, tuple]:
    """

    Parameters
    ----------
    val : np.ndarray
        Values 1d array to fit.
    eq : callable
        Equation to create a fit.

    Returns
    -------
    y_fit : np.ndarray
        1d Fitted values array.
    ss_res_norm : np.ndarray
        Sum of squares of residuals normalized.
    popt : tuple

    """
    t_range = np.arange(tmax) + 1

    if delete_nan:
        t, val = deleteNaN(val,t)
        t, val = np.log10(t), np.log10(val)
    if eq.__name__ == "erfc_exp":
        p0=[1,1,1]
        popt, _ = curve_fit(eq, t, val, p0=p0, maxfev=200_000_000_0)
    else:
        popt, _ = curve_fit(eq, t, val, maxfev=200_000_000_0)
    print(f"{eq.__name__}: {popt}")
    

    y_fit = eq(t_range, *popt)  # full time length
    # y_fit[y_fit < 1] = np.nan  # too small values to be removed
    # y_fit[y_fit > np.max(val) * 2] = np.nan  # too big values removed

    return y_fit


In [3]:

def arr_minimize(arr: np.ndarray, method: str = "median") -> np.ndarray:
    """
    Minimizes 1d array by removing repeats, according to the given method.

    Parameters
    ----------
    arr : np.ndarray
        1d array to be minimized.
    method : str, optional
        'median' or 'average'. The default is 'median'.

    Returns
    -------
    arr1 : np.ndarray
        minimized array.

    """

    search = np.unique(arr)  # arr of unique elements
    search = search[search > 0]  # remove nans

    arr1 = arr.copy()

    for s in search:
        (positions,) = np.where(arr == s)
        if method == "median":
            mid = int(np.median(positions))

        elif method == "average":
            mid = int(np.average(positions))
        elif method == "max":
            mid = int(np.max(positions))
        elif method == "min":
            mid = int(np.min(positions))

        arr1[positions] = np.nan
        arr1[mid] = s  # mid value is kept

    return arr1


def df_minimize(df: pd.DataFrame, **kwargs) -> pd.DataFrame:
    """

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame to be minimized.

    Returns
    -------
    df : pd.DataFrame
        Minimized DataFrame.

    """
    for i in range(len(df.columns)):
        df.iloc[:, i] = arr_minimize(
            df.iloc[:, i], **kwargs
        )  # values minimized and returned

    return df

In [4]:
# values dataframe
df = pd.read_csv("./data/duration_cont.csv", index_col=None)
df = df_minimize(df, method="median")
df[df == 0] = np.nan
df = df.dropna(axis=0, how="all")
df["timestep"] = df.index + 1
df = pl.from_pandas(df)

In [5]:
df.tail()

uniform_250,uniform_350,gaus_250,gaus_350,3.50_60,timestep
f64,f64,f64,f64,f64,i64
null,2.0,null,null,null,1499
null,null,null,2.0,null,1605
null,null,null,null,1.0,1772
null,1.0,null,null,null,2038
null,null,null,1.0,null,2080


In [6]:
def sample_ends_favored_1D(df: pd.DataFrame, n: int = 10) -> pd.DataFrame:
    """sample a dataframe but favor the ends

    Parameters
    ----------
    df : pd.DataFrame

    n : int, optional
        end size, by default 10

    Returns
    -------
    pd.DataFrame
        sampled dataframe
    """


    mid_df = df.iloc[n:-n]
    sampled_df = pd.concat(
        [df.head(n), mid_df.sample(frac=0.20, random_state=42), df.tail(n)],
        ignore_index=True
    )
    return sampled_df

In [7]:
def sample_EF_1D(df: pl.DataFrame, n: int = 10) -> pd.DataFrame:
    """sample a dataframe but favor the ends

    Parameters
    ----------
    df : pd.DataFrame

    n : int, optional
        end size, by default 10

    Returns
    -------
    pd.DataFrame
        sampled dataframe
    """

    

    mid_df = df.slice(n, len(df) - 2 * n)
    sampled_df = pl.concat(
        [df.head(n), mid_df.sample(fraction=0.20, seed=42), df.tail(n)],
        how="vertical",
    ).unpivot(index="timestep", 
              value_name="remaining", 
              variable_name="case")
    
    return sampled_df

In [8]:
def sample_EF_2D(df: pl.DataFrame, n: int = 10) -> pd.DataFrame:

    all_parts = [
        sample_EF_1D(df.select([col, "timestep"]), n=n).with_columns(
            pl.lit(col).alias("case")
        )
        for col in df.columns if col != "timestep"
    ]
    collected = pl.concat(all_parts, how="vertical")

    return collected.drop_nulls()

In [9]:
def add_type_um(df: pl.DataFrame) -> pl.DataFrame:

    return df.with_columns(
        pl.col("case").str.split("_").list.get(0).alias("type"),
        pl.col("case").str.split("_").list.get(1).alias("um"),
    )


In [10]:
df = sample_EF_2D(df, n=40)
df

timestep,case,remaining
i64,str,f64
1,"""uniform_250""",1.192739e6
2,"""uniform_250""",779689.0
3,"""uniform_250""",541090.0
4,"""uniform_250""",389455.0
5,"""uniform_250""",288740.0
…,…,…
996,"""3.50_60""",6.0
1061,"""3.50_60""",5.0
1124,"""3.50_60""",4.0


In [11]:
df = df.with_columns(
    pl.col("case").str.split("_").list.get(0).alias("distribution"),
    pl.col("case").str.split("_").list.get(1).alias("around"),
)

In [12]:
df350 = df.filter(pl.col("case") == "3.50_60")
df350 = df350.with_columns(pl.col("timestep").log1p(),pl.col("remaining").log1p())

In [13]:
df.filter(pl.col("case") == "uniform_350").sort("timestep").tail()

timestep,case,remaining,distribution,around
i64,str,f64,str,str
962,"""uniform_350""",6.0,"""uniform""","""350"""
1026,"""uniform_350""",5.0,"""uniform""","""350"""
1153,"""uniform_350""",4.0,"""uniform""","""350"""
1499,"""uniform_350""",2.0,"""uniform""","""350"""
2038,"""uniform_350""",1.0,"""uniform""","""350"""


In [14]:
df.filter(pl.col("case") == "gaus_350").sort("timestep").tail()

timestep,case,remaining,distribution,around
i64,str,f64,str,str
1168,"""gaus_350""",5.0,"""gaus""","""350"""
1289,"""gaus_350""",4.0,"""gaus""","""350"""
1388,"""gaus_350""",3.0,"""gaus""","""350"""
1605,"""gaus_350""",2.0,"""gaus""","""350"""
2080,"""gaus_350""",1.0,"""gaus""","""350"""


In [15]:
cases = df.select("case").unique().to_series().to_list()
cases

['uniform_250', 'uniform_350', '3.50_60', 'gaus_350', 'gaus_250']

In [16]:
df.select("timestep").to_series().max()

2080

In [17]:
# make fit
tmax = df.select("timestep").to_series().max()
fits_arr = list()
fits = pl.DataFrame().with_columns(
    pl.int_range(1, tmax + 1).alias("timestep"),
)
cases = df.select("case").unique().to_series().to_list()
for c in cases:
    if not c == "3.50_60":
        case_df = df.filter(pl.col("case") == c)
        timestep = case_df.select("timestep").to_numpy()
        data = case_df.select("remaining").to_numpy()
    
        fits = fits.with_columns(
            pl.Series(value_fit(data, timestep, tmax=tmax, eq=tri_exp)).alias(f"{c}_tri.exp"),
            pl.Series(value_fit(data, timestep, tmax=tmax, eq=quad_exp)).alias(f"{c}_quad.exp"),
            pl.Series(value_fit(data, timestep, tmax=tmax, eq=powerlaw)).alias(f"{c}_powerlaw"),
        )

tri_exp: [1.79125251 0.44334128 2.94931516 0.44335208 1.37720562 0.44334974]
quad_exp: [1.56959194 0.44334844 1.66868248 0.44334945 1.44331479 0.4433473
 1.43617724 0.44334528]
powerlaw: [1. 1.]
tri_exp: [2.07970325 0.40110083 2.12498761 0.40111606 1.47600197 0.40110427]
quad_exp: [1.06631121 0.40111002 1.35331366 0.40110559 1.12112096 0.40111844
 2.13996318 0.40110687]
powerlaw: [1. 1.]
tri_exp: [2.83097844 0.86944261 2.64742681 0.86944391 2.90744558 0.86944151]
quad_exp: [2.09633671 0.86943329 2.09645367 0.8694296  2.09651702 0.86943539
 2.09646958 0.86944114]
powerlaw: [1. 1.]
tri_exp: [2.63425075 0.54593644 2.50472735 0.54593814 2.60642101 0.5459262 ]
quad_exp: [1.86604618 0.54594665 1.88526757 0.54592905 1.99881226 0.54592177
 1.99528062 0.54594028]
powerlaw: [1. 1.]


C:\Users\zafi_\AppData\Local\Temp\ipykernel_10632\1016430612.py:16: RuntimeWarning: divide by zero encountered in reciprocal
  return a * x ** (-b)
C:\Users\zafi_\AppData\Local\Temp\ipykernel_10632\1016430612.py:16: RuntimeWarning: divide by zero encountered in power
  return a * x ** (-b)
C:\Users\zafi_\AppData\Local\Temp\ipykernel_10632\1016430612.py:89: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(eq, t, val, maxfev=200_000_000_0)


In [18]:
# make fit
tmax = df.select("timestep").to_series().max()
fits_arr = list()
fits = pl.DataFrame().with_columns(
    pl.int_range(1, tmax + 1).alias("timestep"),
)
cases = df.select("case").unique().to_series().to_list()
for c in cases:
    if not c == "3.50_60":
        case_df = df.filter(pl.col("case") == c)
        timestep = case_df.select("timestep").to_numpy()
        data = case_df.select("remaining").to_numpy()
    
        fits = fits.with_columns(
            pl.Series(value_fit(data, timestep, tmax=tmax, eq=tri_exp)).alias(f"{c}_tri.exp"),
            pl.Series(value_fit(data, timestep, tmax=tmax, eq=erfc_exp)).alias(f"{c}_erfc.exp"),
        )

tri_exp: [1.79125251 0.44334128 2.94931516 0.44335208 1.37720562 0.44334974]
erfc_exp: [  6.69363318  17.66832227 488.63239323]
tri_exp: [2.63425075 0.54593644 2.50472735 0.54593814 2.60642101 0.5459262 ]
erfc_exp: [  7.83237905 -11.4418433  246.1508716 ]
tri_exp: [2.83097844 0.86944261 2.64742681 0.86944391 2.90744558 0.86944151]
erfc_exp: [  7.8248121   19.015893   349.71221738]
tri_exp: [2.07970325 0.40110083 2.12498761 0.40111606 1.47600197 0.40110427]
erfc_exp: [  6.26573661   9.79032768 177.2047102 ]


C:\Users\zafi_\AppData\Local\Temp\ipykernel_10632\1016430612.py:11: RuntimeWarning: overflow encountered in exp
  * np.exp(( np.square(koff) * t) / ks)
C:\Users\zafi_\AppData\Local\Temp\ipykernel_10632\1016430612.py:10: RuntimeWarning: overflow encountered in multiply
  return (a
C:\Users\zafi_\AppData\Local\Temp\ipykernel_10632\1016430612.py:10: RuntimeWarning: invalid value encountered in multiply
  return (a
C:\Users\zafi_\AppData\Local\Temp\ipykernel_10632\1016430612.py:12: RuntimeWarning: invalid value encountered in sqrt
  * erfc(np.sqrt((np.square(koff) * t) / ks))


In [19]:
fits

timestep,gaus_350_tri.exp,gaus_350_erfc.exp,uniform_250_tri.exp,uniform_250_erfc.exp,gaus_250_tri.exp,gaus_250_erfc.exp,uniform_350_tri.exp,uniform_350_erfc.exp
i64,f64,f64,f64,f64,f64,f64,f64,f64
1,3.926898,3.275505,4.486915,4.031026,3.515224,3.310057,3.803668,3.210168
2,2.520611,2.640131,2.599273,3.28306,1.47353,2.598,2.546853,2.612006
3,1.617939,2.28634,1.505761,2.85956,0.617682,2.218376,1.705317,2.273845
4,1.038528,2.049358,0.872288,2.572925,0.258923,1.970611,1.141843,2.04519
5,0.666614,1.875339,0.505317,2.360895,0.108537,1.791897,0.764553,1.876164
…,…,…,…,…,…,…,…,…
2076,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN
2077,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN
2078,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN


In [20]:
fitsm = fits.unpivot(index="timestep",
                     value_name="remaining",
                     variable_name="fit_case")

In [21]:
fitsm = fitsm.with_columns(
    pl.col("fit_case").str.split("_").list.get(0).alias("distribution"),
    pl.col("fit_case").str.split("_").list.get(1).alias("around"),
    pl.col("fit_case").str.split("_").list.get(2).alias("equation")
)

In [22]:
gauss = fitsm.filter(pl.col("distribution")=="gaus",pl.col("remaining").is_not_nan())
uni = fitsm.filter(pl.col("distribution")=="uniform",pl.col("remaining").is_not_nan())

In [23]:
gauss.write_csv('gaus.csv')
gauss.head(12)

timestep,fit_case,remaining,distribution,around,equation
i64,str,f64,str,str,str
1,"""gaus_350_tri.exp""",3.926898,"""gaus""","""350""","""tri.exp"""
2,"""gaus_350_tri.exp""",2.520611,"""gaus""","""350""","""tri.exp"""
3,"""gaus_350_tri.exp""",1.617939,"""gaus""","""350""","""tri.exp"""
4,"""gaus_350_tri.exp""",1.038528,"""gaus""","""350""","""tri.exp"""
5,"""gaus_350_tri.exp""",0.666614,"""gaus""","""350""","""tri.exp"""
…,…,…,…,…,…
8,"""gaus_350_tri.exp""",0.176296,"""gaus""","""350""","""tri.exp"""
9,"""gaus_350_tri.exp""",0.113162,"""gaus""","""350""","""tri.exp"""
10,"""gaus_350_tri.exp""",0.072637,"""gaus""","""350""","""tri.exp"""


In [24]:
df = df.with_columns(pl.col("timestep").log1p(),pl.col("remaining").log1p())
df.sample(12)

timestep,case,remaining,distribution,around
f64,str,f64,str,str
3.295837,"""3.50_60""",8.444407,"""3.50""","""60"""
5.236442,"""uniform_350""",5.583496,"""uniform""","""350"""
5.010635,"""gaus_350""",5.733341,"""gaus""","""350"""
5.351858,"""uniform_350""",5.375278,"""uniform""","""350"""
5.257495,"""uniform_350""",5.560682,"""uniform""","""350"""
…,…,…,…,…
3.610918,"""uniform_250""",7.964851,"""uniform""","""250"""
3.663562,"""gaus_250""",0.693147,"""gaus""","""250"""
3.465736,"""uniform_250""",8.489411,"""uniform""","""250"""


In [25]:
df_gauss =df.filter(pl.col("distribution")=="gaus")

In [26]:
eqs = ["tri.exp","erfc.exp"]

In [32]:
def plot_it(df:pl.DataFrame,fits:pl.DataFrame,color_by):

    return (ggplot()
            + geom_point(data=df, mapping=aes(x="timestep", y="remaining",color=color_by),fill="#dfdfdf",shape=21, stroke=1, size=5)
            + geom_line(data=fits, mapping=aes(x="timestep", y="remaining",color=color_by),size=3,alpha=1)
            + geom_line(data=df350,mapping=aes(x="timestep", y="remaining"),size=2,alpha=0.7, color="black")
            + scale_color_brewer(labels=["1-4kT", "2-5kT"])
            + scale_fill_brewer(labels=["1-4kT", "2-5kT"])
            # + scale_x_log10(format="~e")
            # + scale_y_log10(format=".0~e")
            + theme_classic()
            + ggsize(800,400)
            + lims(x=(0.72, 3.7e3),y=(0.5, 1.1e7))
            + theme(legend_title=element_blank(), 
                    exponent_format="pow",
                    axis_text=element_text(size=20,color="#1f1f1f"),
                    axis_title=element_text(size=21,color="#1f1f1f"),
                    legend_text=element_text(size=20),
                    legend_position=[0.8, 0.8]
                    )
            + labs(x="Duration (a.u.)", y="Occurence (n)")
            
            )

In [33]:
grid_gauss =[
        plot_it(df=df.filter(pl.col("distribution")=="gaus").sort("around"),
                fits=gauss.filter(pl.col("equation")==eq).sort("around"),
                color_by="around") 
                for eq in eqs 
                ]

In [34]:
grid_uni= [
        plot_it(df=df.filter(pl.col("distribution")=="uniform").sort("around"),
                fits=uni.filter(pl.col("equation")==eq).sort("around"),
                color_by="around")+ scale_fill_hue(labels=["1-4kT", "2-5kT"])+scale_color_hue(labels=["1-4kT", "2-5kT"])
                for eq in eqs 
                ]

In [35]:
all = gggrid(grid_gauss+grid_uni,ncol=2)+ggsize(900,700)
all

In [31]:
all.to_png('./fig6__ERFC.png')

'c:\\Users\\zafi_\\paper\\residence2\\5_distributedAffinity\\fig6__ERFC.png'